### Imports

In [1]:
import numpy as np
import pandas as pd
from pyproj import Transformer
from datetime import date,datetime
from bidict import bidict

In [2]:
#Import general library
import bokeh

#Import to handle data parsed to bokeh plots
from bokeh.models import ColumnDataSource

#Import to handle bokeh notebook
from bokeh.io import output_notebook

#Import for the background of the map plot
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN, STAMEN_TONER, WIKIMEDIA

#Import to create plot, show plot, and output plot
from bokeh.plotting import figure, output_file, show

#Import to handle layout of the bokeh plots
from bokeh.layouts import column, row, layout

#Importing the RangeTool used in the Timeline plot
from bokeh.models import RangeTool

#Import to set the range of the Timeline
from bokeh.models import Range1d

#Import for the button and datepicker widget
from bokeh.models import Button, DatePicker

#Import for callback on an event
from bokeh.events import ButtonClick

#Import palette for the color mapper used in geo_map
#Possible other color palettes
# https://colorcet.holoviz.org/user_guide/index.html
import colorcet as cc
palette = cc.CET_L18

#Import palette for the histogram colors
from bokeh.palettes import OrRd3


#Import log_cmap to support the color mapper for the geo_map
from bokeh.transform import log_cmap, linear_cmap

#Importing the select tools for bokeh plots
from bokeh.models import LassoSelectTool, BoxSelectTool

#Import the tools for the color bar
from bokeh.models import ColorBar, LogTicker
#Other option for ticker: AdaptiveTicker

#Import legend to show the legend for the timeline and histogram
from bokeh.models import Legend

#Import for the histogram model
from bokeh.models import FactorRange

#Import for the coloring of the bars in the histogram plot
from bokeh.transform import factor_cmap

#Import to show multiple tabs
from bokeh.models import Panel, Tabs

output_notebook()

Loading BokehJS ...

### Assumptions

In [3]:
#screen resolution
screen_height = 1080
screen_length = 1920

In [4]:
#Modified code from the bidict package
#Source: https://stackoverflow.com/questions/3318625/how-to-implement-an-efficient-bidirectional-hash-table
class bidict(dict):
    def __init__(self, *args, **kwargs):
        super(bidict, self).__init__(*args, **kwargs)
        self.inverse = {}
        for key, value in self.items():
            self.inverse.setdefault(value,[]).append(key) 

    def __setitem__(self, key, value):
        if key in self:
            self.inverse[self[key]].remove(key) 
        super(bidict, self).__setitem__(key, value)
        self.inverse.setdefault(value,[]).append(key)        

    def __delitem__(self, key):
        self.inverse.setdefault(self[key],[]).remove(key)
        if self[key] in self.inverse and not self.inverse[self[key]]: 
            del self.inverse[self[key]]
        super(bidict, self).__delitem__(key)

### Data import and preparation

In [5]:
#Columns that we want to keep from the data we load in
import_columns = ['created_at_CET','latitude','longitude','Overall.score','Anger','Confusion',
                  'Disgust','Fear','Happiness','Sadness','Shame','Surprise']

#Load the data into the python environment
twitter_data = pd.read_csv("D:/Master Project/complete_swiss_dataset.csv",encoding = "ISO-8859-15",low_memory=False)

#Select only the columns that are relevant
#Timeline Map Histogram data (TMH_data)
TMH_data = twitter_data[import_columns].sort_values(by='created_at_CET',ignore_index=True)

#Round all entries in dataframe
#Mainly to remove specific gps locations of users
TMH_data = TMH_data.round(3)

#Format the string input to datetime
TMH_data['created_at_CET'] = pd.to_datetime(TMH_data['created_at_CET'], format='%Y-%m-%d %H:%M:%S')

#Obtain a list of tweets which are within the specified coordinates
geo_bounds = np.where((TMH_data['latitude'] >= 45.8) &
                      (TMH_data['latitude'] <= 47.9) &
                      (TMH_data['longitude'] >= 5.9) &
                      (TMH_data['longitude'] <= 10.6))[0]

#Set the data to only the points within the bounds
TMH_data = TMH_data.iloc[geo_bounds]

#Sort the data by the date at creation
TMH_data = TMH_data.sort_values(by='created_at_CET')

#Removing the entire dataset now that the selection has been made
twitter_data = None

In [6]:
#Declare transformer to convert from epsg4326 to epsg 3856 (web mercator)
transformer_merc = Transformer.from_crs("EPSG:4326", "EPSG:3857")

#Transform the coordinates from epsg 4326 to epsg 3857
mercator_transform = transformer_merc.transform(TMH_data['latitude'].values,
                                                TMH_data['longitude'].values)

#
TMH_data['x_mercator'] = mercator_transform[0].round(1)
TMH_data['y_mercator'] = mercator_transform[1].round(1)

#Declare transformer to convert from epsg4326 to epsg 21781 (swiss coordinate system)
transformer_swiss = Transformer.from_crs("EPSG:4326", "EPSG:21781")

#Transform the coordinates from epsg 4326 to epsg 3857
swiss_transform = transformer_swiss.transform(TMH_data['latitude'].values,
                                                 TMH_data['longitude'].values)

#
TMH_data['x_swiss'] = (swiss_transform[0] / 1e3).round()
TMH_data['y_swiss'] = (swiss_transform[1] / 1e3).round()

TMH_data['ID'] = TMH_data.index

In [ ]:
#Loop the entire dataframe and add element for day and year

dayofyear = []
year = []

for i in range(0,len(TMH_data)):
    dayofyear.append((TMH_data.iloc[i]['created_at_CET']).dayofyear)
    year.append((TMH_data.iloc[i]['created_at_CET']).year)
    
TMH_data['dayofyear'] = dayofyear
TMH_data['year'] = year
dayofyear = None
year = None


In [7]:
#Create a dataframe that can be easily queried by time
time_table = TMH_data[['created_at_CET','ID']]
time_table.index = time_table['created_at_CET']
del time_table['created_at_CET']

In [7]:
# TABLE - timeline_data
# contains the amount of tweets per day and their corresponding tweet IDs

timeline_data = pd.DataFrame()

timeline_data['dates'] = TMH_data['created_at_CET'].copy(deep=True)

timeline_data['ID'] = timeline_data.index

timeline_data['hour_minute'] = timeline_data['dates'].dt.strftime('%H-%m')

timeline_data['dates'] = timeline_data['dates'].dt.strftime('%Y-%j')

del timeline_data['ID']

In [92]:
temp = np.array(TMH_data[['x_mercator','y_mercator','ID']])

In [415]:
time_table['2015']

,ID
created_at_CET,
2015-01-01 01:00:03,0
2015-01-01 01:00:09,1
2015-01-01 01:00:14,2
2015-01-01 01:00:22,3
2015-01-01 01:00:34,4
...,...
2015-12-31 23:58:46,489445
2015-12-31 23:58:50,489446
2015-12-31 23:59:01,489447


In [154]:
#Decalre an empty bidirectional dictionary
loc_link_ID = bidict({})

#Selected the relevant columns from the twitter data
geo_tweet_data = np.array(TMH_data[['x_mercator','y_mercator','ID']])

#Loop all the tweets in the data
for tweet in geo_tweet_data:
    #Set the ID to point to the coordinate in the dictionary
    loc_link_ID[tweet[2]] = str(tweet[0])+" "+str(tweet[1])
    

In [166]:
%%time
loc_link_ID[0]

Wall time: 0 ns


'951559.0 6017154.4'

In [170]:
%%time
loc_link_ID.inverse['728252.1 5866016.3']

Wall time: 0 ns


[13036.0,
 13037.0,
 13038.0,
 13039.0,
 13105.0,
 13107.0,
 13109.0,
 13110.0,
 13113.0]

### Timeline bokeh plot

#### Data preparation for timline

Two lines will be shown on the graph:
1. Total amount of tweets per day
2. Amount of tweets for the selected data


In [8]:
#Function to compute the number of tweets on days
#Input - list of tweet IDs

def tweets_per_day(IDs=None):
    #If no IDs have been passed then return the counts for all dates in the data
    #TODO: link timeline_data to the database table
    if(IDs == None):
        output = np.array(np.unique(timeline_data.iloc[:]['dates'],return_counts=True))
    else:
        output = np.array(np.unique(timeline_data.iloc[IDs]['dates'],return_counts=True))
    
    output[0] = pd.to_datetime(output[0],format='%Y-%j')
    
    return output
    
    

In [14]:
testing = np.empty((2,500))

In [15]:
a  = freq_list[0]
b = np.zeros(len(a))

In [16]:
testing = np.array([a,b])

In [17]:
testing1 = pd.DataFrame(testing.T)
testing1.columns = ['date','freq']

In [158]:
test1,test2 = tweets_per_day([0,1,2,3,4])

In [163]:
temp = tweets_per_day()

In [165]:
temp[0]

array(['2015-001', '2015-002', '2015-003', ..., '2018-247', '2018-248',
       '2018-249'], dtype=object)

In [286]:
time_table['2015-01-01':'2015-01-01']['ID'].values

array([   0,    1,    2, ..., 2959, 2960, 2961], dtype=int64)

In [10]:
def timeline(doc):
        
    #Create the Timeline figure
    Timeline = figure(background_fill_color="#fafafa",
                      x_axis_type='datetime',
                      x_axis_label="Date",
                      #y_axis_type=None,
                      y_axis_label="Amount of tweets",
                      tools="pan,box_zoom,wheel_zoom,reset",
                      toolbar_location="above",
                      plot_width=int(screen_length*0.7),
                      plot_height=int(screen_height*0.3))
    
    total_tweet_counts = tweets_per_day()
    
    #Load the data for the timeline into a ColumnDataSource for the plot
    source_total = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                              freq=total_tweet_counts[1]))
    
    #Declare an empty list to contain the items in the legend
    legend_items = []
    
    #Plot a line on the graph for the total amount of tweets per day
    total_tweets_per_day = Timeline.line('date','freq',source=source_total,line_color="black")
    
    #Add the total tweets to the legend
    legend_items.append(("Total",[total_tweets_per_day]))
    
    #Load the data for the geo-selected line, at initialization its 0 for all dates
    source_selected = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                                 freq=np.zeros(len(total_tweet_counts[1]))))
    
    #Plot the line for the selected tweets
    selected_tweets_per_day = Timeline.line('date','freq',source=source_selected,line_color="blue")
    
    #Add the selected tweets to the legend
    legend_items.append(("Geo selected",[selected_tweets_per_day]))
    
    #Set the selected line to not be visible initially
    selected_tweets_per_day.visible = False
    
    #Add the legend_items to the legend object
    legend = Legend(items = legend_items)
    
    #Set the onclick action of the legend to hide
    legend.click_policy="hide"
    
    Timeline.add_layout(legend,'left')

    
    #Declare a button which will update the values in Map according to the time period
    update_time_button = Button(label="Update Time Period", button_type="success",
                   width = int(screen_length*0.05),
                   height = int(screen_height*0.075))
    
    #Callback to handle the changing x range of the range tool
    #Will update the corresponding start and end dates in the datepickers
    #datepickers: start_date_input and end_date_input
    def range_tool_to_datepicker_handler(attr,old,new):

        
        #If the starting value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.start) == float or
           type(range_tool.x_range.start) == int):
            #Extract the start value of the interval and convert it into correct format for datepicker
            new_start_date = datetime.fromtimestamp(range_tool.x_range.start / 1e3).date()
            
            #Check whether the date is less than the minimum of of the data
            if(new_start_date >= min_date_interval):
                
                #If the start date is later than the global minimum then set it as the new date
                start_date_input.value = new_start_date
            else:
                #If the date is before the global minimum then take the global minimum as the start of the range
                start_date_input.value = min_date_interval

        #If the ending value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.end) == float or
           type(range_tool.x_range.end) == int):
            
            #Extract the end value of the interval and convert it into correct format for datepicker
            new_end_date = datetime.fromtimestamp(range_tool.x_range.end / 1e3).date()
            
            #Check whether the date is less than the maximum of of the data
            if(new_end_date <= max_date_interval):
                #If the new end date is before the global maximum
                #Set the end date to the new end date
                end_date_input.value = new_end_date
            else:
                #If the value is after the global maximum
                #then set the value to the global maximum
                end_date_input.value = max_date_interval

    #Callback for when the datepicker of starting date changes
    #AND button is pressed (due to asynchronus behavior of udpates)
    def date_picker_to_range_tool_handler(event):
        range_tool.x_range.start = datetime.fromisoformat(start_date_input.value).timestamp() * 1e3
        range_tool.x_range.end = datetime.fromisoformat(end_date_input.value).timestamp() * 1e3
        print(time_table[start_date_input.value:end_date_input.value]['ID'].values)
    
    update_time_button.on_click(date_picker_to_range_tool_handler)
    
    #Set the global max and min for the date ranges
    min_date_interval = TMH_data.iloc[0]['created_at_CET'].date()
    max_date_interval = TMH_data.iloc[-1]['created_at_CET'].date()
    
    #Create a datepicker for the start date
    start_date_input = DatePicker(title='Start date', value=min_date_interval,
                            min_date=min_date_interval, max_date=max_date_interval,
                            width = int(screen_length*0.1),
                            height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #start_date_input.on_change('value',date_picker_to_range_tool_handler_start)
    
    #Create a datepicker for the end date
    end_date_input = DatePicker(title='End date', value=max_date_interval,
                          min_date=min_date_interval, max_date=max_date_interval,
                          width = int(screen_length*0.1),
                          height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #end_date_input.on_change('value',date_picker_to_range_tool_handler_end)
    
    #Declare a RangeTool for the selection of the date ranges
    range_tool = RangeTool(x_range=Range1d(start=TMH_data.iloc[0]['created_at_CET'],
                                           end=TMH_data.iloc[-1]['created_at_CET']))
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.1
    
    #Link the range_tool to the callbacks for its interval
    range_tool.x_range.on_change('start',range_tool_to_datepicker_handler)
    range_tool.x_range.on_change('end',range_tool_to_datepicker_handler)
    
    #Add the range_tool to the Timeline plot
    Timeline.add_tools(range_tool)
    Timeline.toolbar.active_multi = range_tool
    
    ## DEVELOPMENT HELP ##
    
    print_button = Button(label="Print", button_type="success",
                   width = int(screen_length*0.05),
                   height = int(screen_height*0.075))
    
    def print_handler(event):
        print(min_date_interval,max_date_interval)
    
    print_button.on_click(print_handler)
    
    ## ##
    
    #Create the layout for the Timeline plot
    plots = layout([
        [Timeline,[[start_date_input,end_date_input],update_time_button]],        
        [print_button]
    ])
    #Add the plot to doc
    doc.add_root(plots)

In [11]:
show(timeline)

### Map bokeh plot

In [9]:
#Function to compute the map points and their densities given a certain timeframe
def compute_map_data(IDs = []):
    
    #Copy the relevant data from the dataframe
    if(len(IDs) == 0):
        #If no IDs have been parsed, then return all elements
        Map_data = TMH_data.iloc[:][['x_mercator','y_mercator']].sort_values(by=['x_mercator','y_mercator']).copy()
    else:
        Map_data = TMH_data.iloc[IDs][['x_mercator','y_mercator']].sort_values(by=['x_mercator','y_mercator']).copy()

    #Reset the index for the calculation of densities
    Map_data = Map_data.set_index(np.arange(0,len(Map_data)))

    #Drop all the duplciates, which keeps the first occuring entry in the dataframe
    Map_data = Map_data.drop_duplicates(subset=['x_mercator','y_mercator'])

    #Declare an array the length of the coordinates without duplicates
    #Will represent the density at each coordinate
    density = np.ones(len(Map_data))

    #Loop the array with all the coordinates
    for i in range(0,len(Map_data)-1):
        density[i] = Map_data.index[i+1] - Map_data.index[i]

    #Add the column to the Map_data
    Map_data['density'] = density
    
    return Map_data

In [504]:
%%time
compute_map_data(time_table['2015':'2015']['ID'].values)

Wall time: 309 ms


,x_mercator,y_mercator,density
0,659568.0,5901200.3,1.0
1,662573.6,5756983.4,1.0
2,663798.1,5789327.8,1.0
3,664577.4,5804098.1,1.0
4,664577.4,5804580.2,3.0
...,...,...,...
489139,1163734.0,5877674.6,1.0
489140,1164067.9,5867634.6,1.0
489141,1165181.1,5930655.8,2.0
489143,1166405.6,5920880.5,1.0


In [505]:
time_table['2015':'2015']['ID'].values

array([     0,      1,      2, ..., 489447, 489448, 489449], dtype=int64)

In [ ]:
Start date
End date

[76725, 76733, 76756, 76759, 76761, 76778, 76783, 76787, 76788, 76791, 76794, 76796, 76798, 76799, 76800, 76802, 76803, 76804, 76805, 76806, 76808, 76809, 76811, 76813, 76814, 76815, 76817, 76818, 76825, 76826, 76827, 76828, 76829, 76831, 76832, 76833, 76834, 76835, 76836, 76837, 76838, 76839, 76840, 76841, 76842, 76843, 76846, 76848, 76849, 76851]
2015-01-01 2018-09-06


In [555]:
%%time
temp1 = compute_map_data()

Wall time: 490 ms


In [574]:
%%time
temp2 = np.array(temp1.iloc[[76833, 76834, 76839, 76842, 76846, 76848, 76849, 76851]][['x_mercator','y_mercator']])

Wall time: 999 µs


In [584]:
%%time
np.where(TMH_data['x_mercator'] == temp2[0][0]) and np.where(TMH_data['y_mercator'] == temp2[0][1])

Wall time: 6 ms


(array([   1507,    1647,    1652,    2019,    2046,    2345,    3686,
           5193,   11287,   15628,   15672,   16167,   16191,   16356,
          22220,   22513,   23321,   24380,   29073,   33938,   39409,
          44549,   44551,   53316,   54767,   56711,   59464,   66845,
          67675,   68182,   68571,   69236,   70503,   74295,   77901,
          82856,   84785,   87793,   88544,   93067,  101371,  101662,
         103471,  110930,  117768,  118512,  121604,  123613,  125809,
         125813,  125814,  127713,  128571,  128957,  129850,  131507,
         132389,  134459,  134488,  134852,  136265,  137350,  137929,
         138087,  139172,  139385,  139617,  139618,  141641,  141994,
         142267,  142297,  142774,  143666,  143912,  145867,  146335,
         146357,  149241,  149541,  151358,  157162,  162113,  163750,
         163752,  163990,  164355,  164528,  164530,  164727,  172426,
         172429,  173485,  173658,  178788,  179059,  181076,  181694,
      

In [572]:
temp2

,x_mercator,y_mercator,density
1115452,1163288.7,5867310.9,1.0
1115453,1163288.7,5868929.5,3.0
1115464,1163734.0,5866501.8,6.0
1115472,1164067.9,5867634.6,18.0
1115496,1165292.4,5870548.3,1.0
1115504,1165515.1,5867796.5,1.0
1115505,1166071.7,5870872.1,1.0
1115507,1167073.5,5873948.8,3.0


In [ ]:
TMH_data.iloc[]

In [213]:
transformer = Transformer.from_crs("epsg:4326","epsg:21781")

In [219]:
transformer.transform([45.8,47.9,45.8],[5.9,5.9,10.6])

([480365.00165947096, 484949.27438847, 845752.8481471653],
 [73213.13486442798, 306635.9163027154, 76990.36746356018])

In [541]:
def geoMap(doc):
    #load the background of the map plot
    tile_provider = get_provider(CARTODBPOSITRON)

    #Create the Map figure
    # range bounds supplied in web mercator coordinates
    geo_map = figure(x_range=(650000, 1180000), y_range=(5700000, 6100000),
                     x_axis_type="mercator", y_axis_type="mercator",
                     plot_width=int(screen_length*0.4),
                     plot_height=int(screen_height*0.6))
    
    #Add the selector tools Lasso and Boxselect
    geo_map.add_tools(LassoSelectTool())
    geo_map.add_tools(BoxSelectTool())
    
    #Add the background to the map
    geo_map.add_tile(tile_provider)
    
    #Load in the map data
    source_map = ColumnDataSource(compute_map_data())
    
    #Initialize the color mapper based on the values of density in the map_datat
    color_mapper = log_cmap(field_name = 'density', palette = palette,
                               low = Map_data['density'].min(),
                               high = Map_data['density'].max())
    
    #Defines color bar which indicates the levels
    color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     ticker = LogTicker(num_minor_ticks=1), 
                     label_standoff = 13, width=20, location=(0,0))# Set color_bar location
    
    #Add the color bar to the figure
    geo_map.add_layout(color_bar, 'left')
    
    #Plot the data points with their color corresponding to the amount of tweets at the location
    geo_map.circle(x='x_mercator', y='y_mercator', size=2, color=color_mapper, alpha=1, source=source_map)
    
    #Declare a button which will update the values in Map according to the time period
    map_update_button = Button(label="Update Time Period Map", button_type="success",
                               width = int(screen_length*0.05),
                               height = int(screen_height*0.075))
    
    #Set the global max and min for the date ranges
    min_date_interval = TMH_data.iloc[0]['created_at_CET'].date()
    max_date_interval = TMH_data.iloc[-1]['created_at_CET'].date()
    
    #Create a datepicker for the start date
    start_date_input = DatePicker(title='Start date', value=min_date_interval,
                            min_date=min_date_interval, max_date=max_date_interval,
                            width = int(screen_length*0.1),
                            height = int(screen_height*0.075))
    
    #Create a datepicker for the end date
    end_date_input = DatePicker(title='End date', value=max_date_interval,
                          min_date=min_date_interval, max_date=max_date_interval,
                          width = int(screen_length*0.1),
                          height = int(screen_height*0.075))
    
    ### INTERACTIVITY (START) ###
    
    def update_map_time_based(event):
        start = start_date_input.value
        end = end_date_input.value
        
        IDs = time_table[start:end]['ID'].values
        #coords = Map_data.iloc[source.selected.indices][['x_mercator','y_mercator']]
        
        new_map_data = compute_map_data(IDs)
        
        color_mapper = log_cmap(field_name = 'density', palette = palette,
                                low = new_map_data['density'].min(),
                                high = new_map_data['density'].max())
        
        color_bar.color_mapper=color_mapper['transform']
        
        source_map.data = new_map_data
        
    map_update_button.on_click(update_map_time_based)
    
    ### INTERACTIVITY (END) ###
    
    plots = layout([
        [geo_map],
        [start_date_input,end_date_input,map_update_button],
        
    ])
    doc.add_root(plots)

In [542]:
show(geoMap)

### Histogram bokeh plot

In [429]:
#Function to find the values for a bar on a plot, given the IDs parsed
def compute_vbar_hist(IDs_selected = [],IDs_total = [], column_list = ['Overall.score','Anger','Confusion','Disgust','Fear','Happiness','Sadness','Shame','Surprise']):
    
    if(len(IDs_selected) == 0):
        data_selected = TMH_data[column_list]
    else:
        data_selected = TMH_data.iloc[IDs_selected][column_list]
    
    if(len(IDs_total) == 0):
        data_total = TMH_data[column_list]
    else:
        data_total = TMH_data.iloc[IDs_total][column_list]
        
    mean_values = np.ones(len(column_list)*2)*-1
    
    #Loop all the columns in column_list
    for i in range(0,len(column_list)):
        mean_values[i*2] = data_selected[column_list[i]].mean()
        mean_values[(i*2)+1] = data_total[column_list[i]].mean()
        
    x = [(column,data_type) for column in column_list for data_type in ["Selected","Total"]]
    
    output = pd.DataFrame()
    
    output['measure'] = x
    output['mean'] = mean_values
    
    return output

In [421]:
TMH_data.iloc[5]

created_at_CET    2015-01-01 01:00:35
latitude                       47.258
longitude                       8.852
Overall.score                       3
Anger                               0
Confusion                           0
Disgust                             0
Fear                                0
Happiness                           3
Sadness                             0
Shame                               0
Surprise                            0
x_mercator                     985400
y_mercator                5.98429e+06
x_swiss                           707
y_swiss                           235
ID                                  5
Name: 5, dtype: object

In [456]:
temp = compute_vbar_hist([1,2,3,4,5,6,7,8,9,10])

In [470]:
temp

,measure,mean
0,"(Overall.score, Selected)",1.200000
1,"(Overall.score, Total)",0.144375
2,"(Anger, Selected)",0.000000
3,"(Anger, Total)",0.004479
4,"(Confusion, Selected)",0.000000
5,"(Confusion, Total)",0.001351
6,"(Disgust, Selected)",0.000000
7,"(Disgust, Total)",0.004659
8,"(Fear, Selected)",0.000000
9,"(Fear, Total)",0.004870


In [482]:
#Function to compute the differences between the selected and the total
def measure_diff_hist(df,measures = ['Overall.score','Anger','Confusion','Disgust','Fear','Happiness','Sadness','Shame','Surprise']):
    
    output_diff = np.zeros(int(len(df)/2))

    for i in range(0,int(len(df)/2)):
        output_diff[i] = ((df.iloc[i*2]['mean'] / df.iloc[(i*2)+1]['mean'])-1)*100
        
    output = pd.DataFrame()
    
    output['measure'] = measures
    output['difference'] = output_diff
    
    return output

In [484]:
measure_diff_hist(temp)

,measure,difference
0,Overall.score,731.169063
1,Anger,-100.000000
2,Confusion,-100.000000
3,Disgust,-100.000000
4,Fear,-100.000000
5,Happiness,1426.356214
6,Sadness,-100.000000
7,Shame,-100.000000
8,Surprise,-100.000000


In [485]:
def H_app(doc):
    
    ###  Histogram (START) ###
    
    measures = ['Overall.score','Anger','Confusion','Disgust','Fear','Happiness','Sadness','Shame','Surprise']
    
    hist_data = compute_vbar_hist(np.arange(0,1000))
    
    source_hist = ColumnDataSource(hist_data)
    
    histogram = figure(plot_width=int(screen_length),plot_height=int(screen_height),
                  x_range=FactorRange(*hist_data['measure'].values),toolbar_location=None, title="Scores per Category",
                      y_axis_label="Score")
    
    histogram.vbar(x='measure', top='mean', width=1, source=source_hist,
                  fill_color=factor_cmap('measure', palette=OrRd3, factors=["Selected","Total"], start=1, end = 2))
    
    histogram.y_range.start = 0
    histogram.x_range.range_padding = 0.1
    histogram.xaxis.major_label_orientation = 1
    histogram.xgrid.grid_line_color = None
    
    tab_hist = Panel(child=histogram, title="Histogram")
    
    ###  Histogram (END) ###
    
    ### Differences (START) ###
    
    differences = figure(plot_width=int(screen_length),plot_height=int(screen_height),
                         x_range=measures,toolbar_location=None,y_axis_label="Percentage difference from overall")
    
    diff_data = measure_diff_hist(hist_data)
    
    source_diff = ColumnDataSource(diff_data)
    
    differences.vbar(x='measure',top='difference', source=source_diff,width=0.9,fill_color=OrRd3[0])
    
    tab_diff = Panel(child=differences, title="Differences")
    
    ### Differences (END) ###
    
    plots = layout([
        [Tabs(tabs=[tab_hist,tab_diff])],
        
    ])
    doc.add_root(plots)

In [486]:
show(H_app)

### Timeline + Map bokeh plots together

In [179]:
def TM_app(doc):
    
    ###  Timeline (START) ###
    
    #Create the Timeline figure
    Timeline = figure(background_fill_color="#fafafa",
                      x_axis_type='datetime',
                      x_axis_label="Date",
                      #y_axis_type="log",
                      y_axis_label="Amount of tweets",
                      tools="pan,box_zoom,wheel_zoom,reset",
                      toolbar_location="above",
                      plot_width=int(screen_length*0.8),
                      plot_height=int(screen_height*0.3))
    
    total_tweet_counts = tweets_per_day()
    
    #Load the data for the timeline into a ColumnDataSource for the plot
    source_total = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                              freq=total_tweet_counts[1]))
    
    #Declare an empty list to contain the items in the legend
    legend_items = []
    
    #Plot a line on the graph for the total amount of tweets per day
    total_tweets_per_day = Timeline.line('date','freq',source=source_total,line_color="black")
    
    #Add the total tweets to the legend
    legend_items.append(("Total",[total_tweets_per_day]))
    
    #Load the data for the geo-selected line, at initialization its 0 for all dates
    source_selected = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                                 freq=np.zeros(len(total_tweet_counts[1]))))
    
    #Plot the line for the selected tweets
    selected_tweets_per_day = Timeline.line('date','freq',source=source_selected,line_color="orange")
    
    #Add the selected tweets to the legend
    legend_items.append(("Geo selected",[selected_tweets_per_day]))
    
    #Set the selected line to not be visible initially
    selected_tweets_per_day.visible = False
    
    #Add the legend_items to the legend object
    legend = Legend(items = legend_items)
    
    #Set the onclick action of the legend to hide
    legend.click_policy="hide"
    
    Timeline.add_layout(legend,'left')

    
    #Declare a button which will update the values in Map according to the time period
    update_time_button = Button(label="Sync Time", button_type="success",
                                width = int(screen_length*0.1),
                                height = int(screen_height*0.075))
    
    #Callback to handle the changing x range of the range tool
    #Will update the corresponding start and end dates in the datepickers
    #datepickers: start_date_input and end_date_input
    def range_tool_to_datepicker_handler(attr,old,new):

        
        #If the starting value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.start) == float or
           type(range_tool.x_range.start) == int):
            #Extract the start value of the interval and convert it into correct format for datepicker
            new_start_date = datetime.fromtimestamp(range_tool.x_range.start / 1e3).date()
            
            #Check whether the date is less than the minimum of of the data
            if(new_start_date >= min_date_interval):
                
                #If the start date is later than the global minimum then set it as the new date
                start_date_input.value = new_start_date
            else:
                #If the date is before the global minimum then take the global minimum as the start of the range
                start_date_input.value = min_date_interval

        #If the ending value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.end) == float or
           type(range_tool.x_range.end) == int):
            
            #Extract the end value of the interval and convert it into correct format for datepicker
            new_end_date = datetime.fromtimestamp(range_tool.x_range.end / 1e3).date()
            
            #Check whether the date is less than the maximum of of the data
            if(new_end_date <= max_date_interval):
                #If the new end date is before the global maximum
                #Set the end date to the new end date
                end_date_input.value = new_end_date
            else:
                #If the value is after the global maximum
                #then set the value to the global maximum
                end_date_input.value = max_date_interval

    #Callback for when the datepicker of starting date changes
    #AND button is pressed (due to asynchronus behavior of udpates)
    def date_picker_to_range_tool_handler(event):
        
        start = start_date_input.value
        end = end_date_input.value
        
        range_tool.x_range.start = datetime.fromisoformat(start).timestamp() * 1e3
        range_tool.x_range.end = datetime.fromisoformat(end).timestamp() * 1e3
                
        IDs = time_table[start:end]['ID'].values
        
        new_map_data = compute_map_data(IDs)
        
        color_mapper = log_cmap(field_name = 'density', palette = palette,
                                low = new_map_data['density'].min(),
                                high = new_map_data['density'].max())
        
        color_bar.color_mapper=color_mapper['transform']
        
        source_map.data = new_map_data
    
    update_time_button.on_click(date_picker_to_range_tool_handler)
    
    #Set the global max and min for the date ranges
    min_date_interval = TMH_data.iloc[0]['created_at_CET'].date()
    max_date_interval = TMH_data.iloc[-1]['created_at_CET'].date()
    
    #Create a datepicker for the start date
    start_date_input = DatePicker(title='Start date', value=min_date_interval,
                            min_date=min_date_interval, max_date=max_date_interval,
                            width = int(screen_length*0.1),
                            height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #start_date_input.on_change('value',date_picker_to_range_tool_handler_start)
    
    #Create a datepicker for the end date
    end_date_input = DatePicker(title='End date', value=max_date_interval,
                          min_date=min_date_interval, max_date=max_date_interval,
                          width = int(screen_length*0.1),
                          height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #end_date_input.on_change('value',date_picker_to_range_tool_handler_end)
    
    #Declare a RangeTool for the selection of the date ranges
    range_tool = RangeTool(x_range=Range1d(start=TMH_data.iloc[0]['created_at_CET'],
                                           end=TMH_data.iloc[-1]['created_at_CET']))
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.1
    
    #Link the range_tool to the callbacks for its interval
    range_tool.x_range.on_change('start',range_tool_to_datepicker_handler)
    range_tool.x_range.on_change('end',range_tool_to_datepicker_handler)
    
    #Add the range_tool to the Timeline plot
    Timeline.add_tools(range_tool)
    Timeline.toolbar.active_multi = range_tool

    ###  Timeline (END) ###
    
    ### Map (START) ###
    
    #load the background of the map plot
    tile_provider = get_provider(CARTODBPOSITRON_RETINA)

    #Create the Map figure
    # range bounds supplied in web mercator coordinates
    geo_map = figure(x_range=(650000, 1180000), y_range=(5700000, 6100000),
                     x_axis_type="mercator", y_axis_type="mercator",
                     plot_width=int(screen_length),
                     plot_height=int(screen_height*0.7))
    
    #Add the selector tools Lasso and Boxselect
    geo_map.add_tools(LassoSelectTool())
    geo_map.add_tools(BoxSelectTool())
    
    #Add the background to the map
    geo_map.add_tile(tile_provider)
    
    #Compute the data for the map
    map_data = compute_map_data()
    
    #Load in the map data
    source_map = ColumnDataSource(map_data)
    
    #Initialize the color mapper based on the values of density in the map_datat
    color_mapper = log_cmap(field_name = 'density', palette = palette,
                               low = map_data['density'].min(),
                               high = map_data['density'].max())
    
    #Defines color bar which indicates the levels
    color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     ticker = LogTicker(num_minor_ticks=1), 
                     label_standoff = 13, width=20, location=(0,0))# Set color_bar location
    
    #Add the color bar to the figure
    geo_map.add_layout(color_bar, 'left')
    
    #Plot the data points with their color corresponding to the amount of tweets at the location
    geo_map.circle(x='x_mercator', y='y_mercator', size=2, color=color_mapper, alpha=1, source=source_map)
    
    ### Map (END) ###
    
    ### MAP TO TIMELINE (START) ###
    
    #Declare a button which will update the values timeline according to the selected points in the map
    update_geo_button = Button(label="Sync Geo", button_type="success",
                                width = int(screen_length*0.1),
                                height = int(screen_height*0.075))
    
    def geo_to_timeline_handler(event):
        print(map_data.iloc[source_map.selected.indices])
        print(start_date_input.value,end_date_input.value)
    
    update_geo_button.on_click(geo_to_timeline_handler)
    
    ### MAP TO TIMELINE (END) ###
    
    plots = layout([
        [Timeline,[start_date_input,end_date_input,update_time_button,update_geo_button]],
        [geo_map],
        
    ])
    doc.add_root(plots)

In [163]:
show(TM_app)

        x_mercator  y_mercator  density
195431    728140.8   5897464.4      1.0
195434    728252.1   5865369.1      1.0
195436    728252.1   5865854.5      1.0
195437    728252.1   5866016.3      9.0
195446    728252.1   5873624.9      1.0
195447    728252.1   5882374.7      1.0
195448    728252.1   5882536.8      1.0
2015-11-20 2016-01-11


In [68]:
tile_provider = get_provider(STAMEN_TONER).apply_theme(0)

AttributeError: 'int' object has no attribute 'keys'

In [66]:
tile_provider.apply_theme(property_values='dark minimal')

AttributeError: 'str' object has no attribute 'keys'

### Adding all the bokeh plots together

In [489]:
def TMH_app(doc):
    
    ###  Timeline (START) ###
    
    #Create the Timeline figure
    Timeline = figure(background_fill_color="#fafafa",
                      x_axis_type='datetime',
                      x_axis_label="Date",
                      #y_axis_type="log",
                      y_axis_label="Amount of tweets",
                      tools="pan,box_zoom,wheel_zoom,reset",
                      toolbar_location="above",
                      plot_width=int(screen_length*0.8),
                      plot_height=int(screen_height*0.3))
    
    total_tweet_counts = tweets_per_day()
    
    #Load the data for the timeline into a ColumnDataSource for the plot
    source_total = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                              freq=total_tweet_counts[1]))
    
    #Declare an empty list to contain the items in the legend
    legend_items = []
    
    #Plot a line on the graph for the total amount of tweets per day
    total_tweets_per_day = Timeline.line('date','freq',source=source_total,line_color="black")
    
    #Add the total tweets to the legend
    legend_items.append(("Total",[total_tweets_per_day]))
    
    #Load the data for the geo-selected line, at initialization its 0 for all dates
    source_selected = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                                 freq=np.zeros(len(total_tweet_counts[1]))))
    
    #Plot the line for the selected tweets
    selected_tweets_per_day = Timeline.line('date','freq',source=source_selected,line_color="orange")
    
    #Add the selected tweets to the legend
    legend_items.append(("Geo selected",[selected_tweets_per_day]))
    
    #Set the selected line to not be visible initially
    selected_tweets_per_day.visible = False
    
    #Add the legend_items to the legend object
    legend = Legend(items = legend_items)
    
    #Set the onclick action of the legend to hide
    legend.click_policy="hide"
    
    Timeline.add_layout(legend,'left')

    
    #Declare a button which will update the values in Map according to the time period
    update_time_button = Button(label="Sync Time", button_type="success",
                                width = int(screen_length*0.1),
                                height = int(screen_height*0.075))
    
    #Callback to handle the changing x range of the range tool
    #Will update the corresponding start and end dates in the datepickers
    #datepickers: start_date_input and end_date_input
    def range_tool_to_datepicker_handler(attr,old,new):

        
        #If the starting value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.start) == float or
           type(range_tool.x_range.start) == int):
            #Extract the start value of the interval and convert it into correct format for datepicker
            new_start_date = datetime.fromtimestamp(range_tool.x_range.start / 1e3).date()
            
            #Check whether the date is less than the minimum of of the data
            if(new_start_date >= min_date_interval):
                
                #If the start date is later than the global minimum then set it as the new date
                start_date_input.value = new_start_date
            else:
                #If the date is before the global minimum then take the global minimum as the start of the range
                start_date_input.value = min_date_interval

        #If the ending value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.end) == float or
           type(range_tool.x_range.end) == int):
            
            #Extract the end value of the interval and convert it into correct format for datepicker
            new_end_date = datetime.fromtimestamp(range_tool.x_range.end / 1e3).date()
            
            #Check whether the date is less than the maximum of of the data
            if(new_end_date <= max_date_interval):
                #If the new end date is before the global maximum
                #Set the end date to the new end date
                end_date_input.value = new_end_date
            else:
                #If the value is after the global maximum
                #then set the value to the global maximum
                end_date_input.value = max_date_interval

    #Callback for when the datepicker of starting date changes
    #AND button is pressed (due to asynchronus behavior of udpates)
    def date_picker_to_range_tool_handler(event):
        
        start = start_date_input.value
        end = end_date_input.value
        
        range_tool.x_range.start = datetime.fromisoformat(start).timestamp() * 1e3
        range_tool.x_range.end = datetime.fromisoformat(end).timestamp() * 1e3
                
        IDs = time_table[start:end]['ID'].values
        
        #Compute the new points on the map
        new_map_data = compute_map_data(IDs)
        
        color_mapper = log_cmap(field_name = 'density', palette = palette,
                                low = new_map_data['density'].min(),
                                high = new_map_data['density'].max())
        
        color_bar.color_mapper=color_mapper['transform']
        
        source_map.data = new_map_data
        
        new_hist_data = compute_vbar_hist(IDs)
        
        source_hist.data = new_hist_data
        
        new_diff_data = measure_diff_hist(new_hist_data)
        
        source_diff.data = new_diff_data
        
        
    
    update_time_button.on_click(date_picker_to_range_tool_handler)
    
    #Set the global max and min for the date ranges
    min_date_interval = TMH_data.iloc[0]['created_at_CET'].date()
    max_date_interval = TMH_data.iloc[-1]['created_at_CET'].date()
    
    #Create a datepicker for the start date
    start_date_input = DatePicker(title='Start date', value=min_date_interval,
                            min_date=min_date_interval, max_date=max_date_interval,
                            width = int(screen_length*0.1),
                            height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #start_date_input.on_change('value',date_picker_to_range_tool_handler_start)
    
    #Create a datepicker for the end date
    end_date_input = DatePicker(title='End date', value=max_date_interval,
                          min_date=min_date_interval, max_date=max_date_interval,
                          width = int(screen_length*0.1),
                          height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #end_date_input.on_change('value',date_picker_to_range_tool_handler_end)
    
    #Declare a RangeTool for the selection of the date ranges
    range_tool = RangeTool(x_range=Range1d(start=TMH_data.iloc[0]['created_at_CET'],
                                           end=TMH_data.iloc[-1]['created_at_CET']))
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.1
    
    #Link the range_tool to the callbacks for its interval
    range_tool.x_range.on_change('start',range_tool_to_datepicker_handler)
    range_tool.x_range.on_change('end',range_tool_to_datepicker_handler)
    
    #Add the range_tool to the Timeline plot
    Timeline.add_tools(range_tool)
    Timeline.toolbar.active_multi = range_tool

    ###  Timeline (END) ###
    
    ### Map (START) ###
    
    #load the background of the map plot
    tile_provider = get_provider(CARTODBPOSITRON_RETINA)

    #Create the Map figure
    # range bounds supplied in web mercator coordinates
    geo_map = figure(x_range=(650000, 1180000), y_range=(5700000, 6100000),
                     x_axis_type="mercator", y_axis_type="mercator",
                     plot_width=int(screen_length*0.5),
                     plot_height=int(screen_height*0.7))
    
    #Add the selector tools Lasso and Boxselect
    geo_map.add_tools(LassoSelectTool())
    geo_map.add_tools(BoxSelectTool())
    
    #Add the background to the map
    geo_map.add_tile(tile_provider)
    
    #Compute the data for the map
    map_data = compute_map_data()
    
    #Load in the map data
    source_map = ColumnDataSource(map_data)
    
    #Initialize the color mapper based on the values of density in the map_datat
    color_mapper = log_cmap(field_name = 'density', palette = palette,
                               low = map_data['density'].min(),
                               high = map_data['density'].max())
    
    #Defines color bar which indicates the levels
    color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     ticker = LogTicker(num_minor_ticks=1), 
                     label_standoff = 13, width=20, location=(0,0))# Set color_bar location
    
    #Add the color bar to the figure
    geo_map.add_layout(color_bar, 'left')
    
    #Plot the data points with their color corresponding to the amount of tweets at the location
    geo_map.circle(x='x_mercator', y='y_mercator', size=2, color=color_mapper, alpha=1, source=source_map)
    
    ### Map (END) ###
    
    ### MAP TO TIMELINE (START) ###
    
    #Declare a button which will update the values timeline according to the selected points in the map
    update_geo_button = Button(label="Sync Geo", button_type="success",
                                width = int(screen_length*0.1),
                                height = int(screen_height*0.075))
    
    def geo_to_timeline_handler(event):
        #print(map_data.iloc[source_map.selected.indices])
        #print(start_date_input.value,end_date_input.value)
        print(source_hist.data)
    
    update_geo_button.on_click(geo_to_timeline_handler)
    
    ### MAP TO TIMELINE (END) ###
    
    ###  Histogram (START) ###
    
    measures = ['Overall.score','Anger','Confusion','Disgust','Fear','Happiness','Sadness','Shame','Surprise']
    
    hist_data = compute_vbar_hist()
    
    source_hist = ColumnDataSource(hist_data)
    
    histogram = figure(plot_width=int(screen_length*0.5),plot_height=int(screen_height*0.7),
                  x_range=FactorRange(*hist_data['measure'].values),toolbar_location=None, title="Scores per Category",
                      y_axis_label="Score")
    
    histogram.vbar(x='measure', top='mean', width=1, source=source_hist,
                  fill_color=factor_cmap('measure', palette=OrRd3, factors=["Selected","Total"], start=1, end = 2))
    
    histogram.y_range.start = 0
    histogram.x_range.range_padding = 0.1
    histogram.xaxis.major_label_orientation = 1
    histogram.xgrid.grid_line_color = None
    
    tab_hist = Panel(child=histogram, title="Histogram")
    
    ###  Histogram (END) ###
    
    ### Differences (START) ###
    
    differences = figure(plot_width=int(screen_length*0.5),plot_height=int(screen_height*0.7),
                         x_range=measures,toolbar_location=None,y_axis_label="Percentage difference from overall")
    
    diff_data = measure_diff_hist(hist_data)
    
    source_diff = ColumnDataSource(diff_data)
    
    differences.vbar(x='measure',top='difference', source=source_diff,width=0.9,fill_color=OrRd3[0])
    
    tab_diff = Panel(child=differences, title="Differences")
    
    ### Differences (END) ###
    

    plots = layout([
        [Timeline,[start_date_input,end_date_input,update_time_button,update_geo_button]],
        [geo_map,Tabs(tabs=[tab_hist,tab_diff])],
        
    ])
    doc.add_root(plots)

In [490]:
show(TMH_app)

In [ ]:
datetime.datetime(TMH_data['created_at_CET'][0])



In [ ]:
%%time
TMH_data.sort_values(by='created_at_CET')

In [ ]:
%%time
TMH_data.sort_values(by='created_at_CET')

In [ ]:
TMH_data['created_at_CET'] = pd.to_datetime(TMH_data['created_at_CET'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
Range1d(start=TMH_data.iloc[0]['created_at_CET'],end=TMH_data.iloc[0-1]['created_at_CET'])

In [ ]:
TMH_data.iloc[0]['created_at_CET']

In [405]:
temp = {'measure': [('Overall.score', 'Selected'), ('Overall.score', 'Total'), ('Anger', 'Selected'), ('Anger', 'Total'), ('Confusion', 'Selected'), ('Confusion', 'Total'), ('Disgust', 'Selected'), ('Disgust', 'Total'), ('Fear', 'Selected'), ('Fear', 'Total'), ('Happiness', 'Selected'), ('Happiness', 'Total'), ('Sadness', 'Selected'), ('Sadness', 'Total'), ('Shame', 'Selected'), ('Shame', 'Total'), ('Surprise', 'Selected'), ('Surprise', 'Total')], 'mean': (0.14437495965977953, 0.14437495965977953, 0.0044786609198287424, 0.0044786609198287424, 0.0013509491605648348, 0.0013509491605648348, 0.004658847237860283, 0.004658847237860283, 0.004869512833568801, 0.004869512833568801, 0.07861860741973192, 0.07861860741973192, 0.024657735640674416, 0.024657735640674416, 0.0016405022984961166, 0.0016405022984961166, 0.02456988360501725, 0.02456988360501725)}


In [412]:
temp

{'measure': [('Overall.score', 'Selected'),
  ('Overall.score', 'Total'),
  ('Anger', 'Selected'),
  ('Anger', 'Total'),
  ('Confusion', 'Selected'),
  ('Confusion', 'Total'),
  ('Disgust', 'Selected'),
  ('Disgust', 'Total'),
  ('Fear', 'Selected'),
  ('Fear', 'Total'),
  ('Happiness', 'Selected'),
  ('Happiness', 'Total'),
  ('Sadness', 'Selected'),
  ('Sadness', 'Total'),
  ('Shame', 'Selected'),
  ('Shame', 'Total'),
  ('Surprise', 'Selected'),
  ('Surprise', 'Total')],
 'mean': (0.14437495965977953,
  0.14437495965977953,
  0.0044786609198287424,
  0.0044786609198287424,
  0.0013509491605648348,
  0.0013509491605648348,
  0.004658847237860283,
  0.004658847237860283,
  0.004869512833568801,
  0.004869512833568801,
  0.07861860741973192,
  0.07861860741973192,
  0.024657735640674416,
  0.024657735640674416,
  0.0016405022984961166,
  0.0016405022984961166,
  0.02456988360501725,
  0.02456988360501725)}